In [11]:
import pandas as pd
import numpy as np

In [1481]:
hotels = pd.read_excel('hotels.xlsx')

In [203]:
guests = pd.read_excel('guests.xlsx')

In [33]:
preferences = pd.read_excel('preferences.xlsx')

In [1483]:
hotels = hotels.iloc[:, 1:]

In [37]:
guests = guests.iloc[:, 1:]

In [43]:
preferences = preferences.iloc[:, 1:]

In [209]:
guests.rename(columns={'Unnamed: 0': 'index'}, inplace=True)

In [347]:
hotels[hotels['rooms'] == 0] #just in case

,hotel,rooms,price


In [55]:
hotels.head()

,hotel,rooms,price
0,hotel_1,13,273
1,hotel_2,18,92
2,hotel_3,12,141
3,hotel_4,18,157
4,hotel_5,7,298


In [165]:
hotels_new = pd.DataFrame(columns = ['room_number', 'hotel_number'])

In [175]:
hotels[hotels['hotel'] == 'hotel_2']['rooms'].values[0]

18

In [177]:
hotels_new = pd.DataFrame(columns = ['room_number', 'hotel_number'])
for i in hotels['hotel']:
    room_numbers = hotels[hotels['hotel'] == i]['rooms'].values[0]
    a = 0
    for j in range(room_numbers):
        a +=1 
        hotels_new.loc[len(hotels_new)] = [a, i]
    hotels_new

In [213]:
hotels_new = pd.DataFrame(columns = ['room_number', 'hotel_number', 'price'])
for i in hotels['hotel']:
    room_numbers = hotels[hotels['hotel'] == i]['rooms'].values[0]
    price = hotels[hotels['hotel'] == i]['price'].values[0]
    a = 0
    for j in range(room_numbers):
        a +=1 
        hotels_new.loc[len(hotels_new)] = [a, i, price]
    hotels_new

In [215]:
random_numbers = np.random.permutation(len(hotels_new))

In [217]:
hotels_new['random_number'] = random_numbers

In [219]:
hotels_new2 = hotels_new[hotels_new['random_number'] <= 3999]

In [223]:
hotels_new2

,room_number,hotel_number,price,random_number
0,1,hotel_1,273,679
2,3,hotel_1,273,1514
3,4,hotel_1,273,1057
4,5,hotel_1,273,2541
6,7,hotel_1,273,161
...,...,...,...,...
4612,10,hotel_400,145,1100
4613,11,hotel_400,145,1006
4614,12,hotel_400,145,241
4615,13,hotel_400,145,1298


In [231]:
guests.dtypes

index         int64
guest        object
discount    float64
dtype: object

In [233]:
hotels_new2.dtypes

room_number       int64
hotel_number     object
price             int64
random_number     int32
dtype: object

In [255]:
merged_distribution.dtypes

room_number        int64
hotel_number      object
price              int64
random_number      int32
index              int64
guest             object
discount         float64
dtype: object

In [239]:
merged_distribution = pd.merge(hotels_new2, guests, left_on = 'random_number', right_on = 'index', how = 'left')

In [259]:
result_distribution = merged_distribution[['index', 'hotel_number', 'room_number', 'price', 'guest', 'discount']]

In [263]:
result_distribution['price_final'] = result_distribution['price'] * (1 - result_distribution['discount'])

In [269]:
result_distribution_group = result_distribution.pivot_table(values='price_final', index='hotel_number', aggfunc='sum')

In [289]:
hotels_new_price_order = hotels_new.sort_values(by='price', ascending=False)

In [293]:
hotels_new_price_order = hotels_new_price_order.head(4000)

In [299]:
random_numbers2 = np.random.permutation(len(hotels_new_price_order))

In [301]:
hotels_new_price_order['random_number'] = random_numbers2

C:\Users\Mi\AppData\Local\Temp\ipykernel_18604\1549147738.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotels_new_price_order['random_number'] = random_numbers2


In [313]:
merged_distribution_order = pd.merge(hotels_new_price_order, guests, left_on = 'random_number', right_on = 'index', how = 'left')

In [315]:
result_distribution_order = merged_distribution_order[['index', 'hotel_number', 'room_number', 'price', 'guest', 'discount']]

In [317]:
result_distribution_order['price_final'] = result_distribution_order['price'] * (1 - result_distribution_order['discount'])

In [319]:
result_distribution_order_group = result_distribution_order.pivot_table(values='price_final', index='hotel_number', aggfunc='sum')

In [283]:
result_distribution_group

,price_final
hotel_number,
hotel_1,2492.49
hotel_10,1860.36
hotel_100,2140.80
hotel_101,1865.98
hotel_102,963.60
...,...
hotel_95,1306.78
hotel_96,2088.32
hotel_97,4150.08


In [321]:
result_distribution_order_group

,price_final
hotel_number,
hotel_1,3221.40
hotel_10,2013.54
hotel_100,2747.20
hotel_101,2166.18
hotel_103,1725.56
...,...
hotel_95,1251.11
hotel_96,2263.04
hotel_97,4193.28


In [325]:
sum(result_distribution_group['price_final'])

641419.7

In [327]:
sum(result_distribution_order_group['price_final'])

703769.93

In [410]:
result_distribution

,index,hotel_number,room_number,price,guest,discount,price_final
0,679,hotel_1,1,273,guest_680,0.00,273.00
1,1514,hotel_1,3,273,guest_1515,0.07,253.89
2,1057,hotel_1,4,273,guest_1058,0.19,221.13
3,2541,hotel_1,5,273,guest_2542,0.09,248.43
4,161,hotel_1,7,273,guest_162,0.10,245.70
...,...,...,...,...,...,...,...
3995,1100,hotel_400,10,145,guest_1101,0.12,127.60
3996,1006,hotel_400,11,145,guest_1007,0.10,130.50
3997,241,hotel_400,12,145,guest_242,0.11,129.05
3998,1298,hotel_400,13,145,guest_1299,0.13,126.15


Part 2

In [356]:
start_df = preferences[preferences['priority'] == 1]

In [482]:
start_df['rn'] = start_df.groupby('hotel').cumcount() + 1

C:\Users\Mi\AppData\Local\Temp\ipykernel_18604\3157429541.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_df['rn'] = start_df.groupby('hotel').cumcount() + 1


In [488]:
df_hotels = pd.merge(start_df, hotels, on = 'hotel', how = 'left')

In [492]:
df_full = pd.merge(df_hotels, guests, on = 'guest', how = 'left')

In [506]:
guests_first_priority = df_full[df_full['rn'] <= df_full['rooms']]

In [629]:
guests_second_priority = df_full[df_full['rn'] > df_full['rooms']]

In [ ]:
guests_first_priority[guests_first_priority['hotel'] == 'hotel_1'] #just check everything is alright

In [562]:
hotels_rest = guests_first_priority.pivot_table(values= ['guest', 'rooms'], index='hotel', aggfunc={'guest':'count', 'rooms':'mean'})

In [570]:
hotels_rest['rooms'] = hotels_rest['rooms'].astype(int) #можно не делать в принципе

In [578]:
hotels_rest['rest'] = hotels_rest['rooms'] - hotels_rest['guest']

Снова идем по вторым приоритетам для оставшихся 602 гостей с учетом заселенных номеров

In [631]:
guests_second_priority = pd.merge(guests_second_priority, preferences.query('priority == 2'), on = 'guest', how = 'left')

In [633]:
guests_second_priority = pd.merge(guests_second_priority, hotels_rest, left_on = 'hotel_y', right_on = 'hotel', how = 'left')

In [637]:
guests_second_priority['rn_2'] = guests_second_priority.groupby('hotel_y').cumcount() + 1

In [639]:
guests_third_priority = guests_second_priority[guests_second_priority['rn_2'] > guests_second_priority['rest']] #осталось расселить еще 281 гостя

In [641]:
guests_second_priority = guests_second_priority[guests_second_priority['rn_2'] <= guests_second_priority['rest']] #расселены еще 308 гостей

Делаем цикл, потому что непонятно сколько еще таких шагов предстоит пройти

In [762]:
list_priorities = preferences['priority'].unique().tolist()

In [904]:
checked_in_guests = pd.DataFrame()
hotels_rest = hotels
not_checked_in_guests = guests

In [906]:
start_df = preferences[(preferences['priority'] == 1) & preferences['guest'].isin(not_checked_in_guests['guest']) ]
start_df

,guest,hotel,priority
0,guest_1,hotel_168,1
18,guest_2,hotel_369,1
60,guest_3,hotel_157,1
86,guest_4,hotel_235,1
132,guest_5,hotel_121,1
...,...,...,...
99380,guest_3996,hotel_350,1
99424,guest_3997,hotel_19,1
99452,guest_3998,hotel_160,1
99489,guest_3999,hotel_277,1


In [908]:
start_df['rn'] = start_df.groupby('hotel').cumcount() + 1
start_df

C:\Users\Mi\AppData\Local\Temp\ipykernel_18604\3362773801.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_df['rn'] = start_df.groupby('hotel').cumcount() + 1


,guest,hotel,priority,rn
0,guest_1,hotel_168,1,1
18,guest_2,hotel_369,1,1
60,guest_3,hotel_157,1,1
86,guest_4,hotel_235,1,1
132,guest_5,hotel_121,1,1
...,...,...,...,...
99380,guest_3996,hotel_350,1,10
99424,guest_3997,hotel_19,1,4
99452,guest_3998,hotel_160,1,14
99489,guest_3999,hotel_277,1,15


In [910]:
df_hotels = pd.merge(start_df, hotels_rest, on = 'hotel', how = 'left')
df_hotels

,guest,hotel,priority,rn,rooms,price
0,guest_1,hotel_168,1,1,13,298
1,guest_2,hotel_369,1,1,6,133
2,guest_3,hotel_157,1,1,6,65
3,guest_4,hotel_235,1,1,5,136
4,guest_5,hotel_121,1,1,15,56
...,...,...,...,...,...,...
3995,guest_3996,hotel_350,1,10,12,56
3996,guest_3997,hotel_19,1,4,10,113
3997,guest_3998,hotel_160,1,14,11,68
3998,guest_3999,hotel_277,1,15,18,281


In [912]:
checked_in_guests = pd.concat([checked_in_guests, df_hotels[df_hotels['rn'] <= df_hotels['rooms']]], ignore_index=True)
checked_in_guests

,guest,hotel,priority,rn,rooms,price
0,guest_1,hotel_168,1,1,13,298
1,guest_2,hotel_369,1,1,6,133
2,guest_3,hotel_157,1,1,6,65
3,guest_4,hotel_235,1,1,5,136
4,guest_5,hotel_121,1,1,15,56
...,...,...,...,...,...,...
3393,guest_3993,hotel_359,1,8,14,163
3394,guest_3994,hotel_58,1,10,15,120
3395,guest_3996,hotel_350,1,10,12,56
3396,guest_3997,hotel_19,1,4,10,113


In [914]:
checked_in_guests[checked_in_guests['rn'] == 18]

,guest,hotel,priority,rn,rooms,price
3326,guest_3885,hotel_394,1,18,19,291


In [916]:
checked_in_guests[checked_in_guests['hotel'] == 'hotel_394']

,guest,hotel,priority,rn,rooms,price
142,guest_143,hotel_394,1,1,19,291
226,guest_227,hotel_394,1,2,19,291
335,guest_336,hotel_394,1,3,19,291
483,guest_484,hotel_394,1,4,19,291
585,guest_588,hotel_394,1,5,19,291
623,guest_626,hotel_394,1,6,19,291
764,guest_767,hotel_394,1,7,19,291
1030,guest_1036,hotel_394,1,8,19,291
1237,guest_1250,hotel_394,1,9,19,291
1896,guest_1950,hotel_394,1,10,19,291


In [918]:
hotels_rest = checked_in_guests.pivot_table(values= ['guest', 'rooms', 'price'], index='hotel', aggfunc={'guest':'count', 'rooms':'mean', 'price':'mean'})

In [920]:
hotels_rest = hotels_rest.reset_index()

In [932]:
hotels_rest[hotels_rest['hotel'] == 'hotel_394']

,hotel,guests,price,rooms,rest
327,hotel_394,19,291.0,19.0,0.0


In [926]:
hotels_rest['rest'] = hotels_rest['rooms'] - hotels_rest['guest']

In [856]:
hotels_rest = hotels_rest.rename(columns={'guest': 'guests'})

In [996]:
checked_in_guests[checked_in_guests['rn'] == 2]

,guest,hotel,priority,rn,rooms,price,rest,guests
10,guest_11,hotel_157,1,2,6.0,65.0,6.0,0
17,guest_18,hotel_215,1,2,9.0,133.0,9.0,0
38,guest_39,hotel_121,1,2,15.0,56.0,15.0,0
84,guest_85,hotel_266,1,2,18.0,169.0,18.0,0
85,guest_86,hotel_277,1,2,18.0,281.0,18.0,0
...,...,...,...,...,...,...,...,...
3883,guest_3833,hotel_122,4,2,19.0,167.0,11.0,8
3885,guest_3879,hotel_339,4,2,10.0,74.0,2.0,8
3887,guest_3932,hotel_46,4,2,17.0,214.0,6.0,11
3894,guest_1951,hotel_237,5,2,17.0,249.0,5.0,12


In [990]:
checked_in_guests = pd.DataFrame()
hotels_rest = hotels
hotels_rest['rest'] = hotels['rooms']
hotels_rest['guests'] = 0
not_checked_in_guests = guests
while not not_checked_in_guests.empty:
    for priority in list_priorities:
        start_df = preferences[(preferences['priority'] == priority) & preferences['guest'].isin(not_checked_in_guests['guest']) ]
        start_df['rn'] = start_df.groupby('hotel').cumcount() + 1
        df_hotels = pd.merge(start_df, hotels_rest, on = 'hotel', how = 'left')
        checked_in_guests = pd.concat([checked_in_guests, df_hotels[df_hotels['rn'] <= df_hotels['rest']]], ignore_index=True)
        not_checked_in_guests = df_hotels[df_hotels['rn'] > df_hotels['rest']]
        
        hotels_rest = checked_in_guests.pivot_table(values= ['guest', 'rooms', 'price'], index='hotel', aggfunc={'guest':'count', 'rooms':'mean', 'price':'mean'})
        hotels_rest = hotels_rest.rename(columns={'guest': 'guests'})
        hotels_rest['rest'] = hotels_rest['rooms'] - hotels_rest['guests']
        hotels_rest = hotels_rest.reset_index()
        print(priority, len(checked_in_guests))

C:\Users\Mi\AppData\Local\Temp\ipykernel_18604\1903805619.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_df['rn'] = start_df.groupby('hotel').cumcount() + 1
C:\Users\Mi\AppData\Local\Temp\ipykernel_18604\1903805619.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_df['rn'] = start_df.groupby('hotel').cumcount() + 1
C:\Users\Mi\AppData\Local\Temp\ipykernel_18604\1903805619.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

1 3398
2 3706
3 3828
4 3890
5 3924
6 3941
7 3952
8 3959
9 3964
10 3966


C:\Users\Mi\AppData\Local\Temp\ipykernel_18604\1903805619.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_df['rn'] = start_df.groupby('hotel').cumcount() + 1
C:\Users\Mi\AppData\Local\Temp\ipykernel_18604\1903805619.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  start_df['rn'] = start_df.groupby('hotel').cumcount() + 1
C:\Users\Mi\AppData\Local\Temp\ipykernel_18604\1903805619.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_i

11 3967
12 3971
13 3971
14 3971
15 3972
16 3972
17 3972
18 3972
19 3972
20 3972
21 3972
22 3972
23 3972
24 3972
25 3972
26 3972
27 3972
28 3972
29 3972
30 3972
31 3972
32 3972
33 3972
34 3972
35 3972
36 3972
37 3972
38 3972
39 3972
40 3972
41 3972
42 3972
43 3972
44 3972
45 3972
46 3972
47 3972
48 3972
49 3972


In [1000]:
check_df = preferences[~preferences['guest'].isin(checked_in_guests['guest'])] #these guests will be distributed according to the cheapest hotels

In [1032]:
guests_leftovers = check_df['guest'].unique().tolist()

In [1088]:
x = len(guests_leftovers)

28

In [1131]:
hotels_for_leftovers = hotels_rest[hotels_rest['rest'] > 0].sort_values('price') #additional marketing step to let leftovers check in in the cheapest hotels

,hotel,guests,price,rooms,rest
344,hotel_49,12,50.0,19.0,7.0
247,hotel_321,8,54.0,15.0,7.0
295,hotel_365,13,54.0,14.0,1.0
145,hotel_23,10,54.0,15.0,5.0
75,hotel_167,8,55.0,9.0,1.0
...,...,...,...,...,...
162,hotel_245,11,290.0,17.0,6.0
138,hotel_223,10,291.0,17.0,7.0
269,hotel_341,11,298.0,12.0,1.0
94,hotel_184,7,298.0,8.0,1.0


In [1076]:
rooms_leftovers = pd.merge(hotels_new, hotels_for_leftovers, left_on = 'hotel_number', right_on = 'hotel', how = 'inner')

In [1080]:
rooms_leftovers = rooms_leftovers[rooms_leftovers['room_number'] > rooms_leftovers['guests']]

In [1112]:
rooms_leftovers = rooms_leftovers.sort_values('price_x')[:x]

In [1118]:
rooms_leftovers['guest'] = guests_leftovers

In [1142]:
rooms_leftovers['priority'] = 0

In [1166]:
rooms_leftovers = rooms_leftovers.rename(columns={'price_y': 'price'})

In [1148]:
checked_in_guests = checked_in_guests[['guest', 'hotel', 'priority', 'price', 'rooms']]

In [1150]:
rooms_leftovers = rooms_leftovers[['guest', 'hotel', 'priority', 'price', 'rooms']]

In [1168]:
final_guests = pd.concat([checked_in_guests, rooms_leftovers], ignore_index=True)

In [1174]:
final_guests = pd.merge(final_guests, guests, on = 'guest', how = 'left')

In [1180]:
final_guests['price_final'] = final_guests['price'] * (1 - final_guests['discount'])

In [1182]:
result_distribution_group_part2 = final_guests.pivot_table(values='price_final', index='hotel', aggfunc='sum')

In [1184]:
result_distribution_group_part2

,price_final
hotel,
hotel_1,3158.61
hotel_10,2062.38
hotel_100,1704.00
hotel_101,2164.60
hotel_102,1015.43
...,...
hotel_95,1350.73
hotel_96,2281.76
hotel_97,4129.92


In [1186]:
sum(result_distribution_group_part2['price_final'])

644159.43

Part 3

In [ ]:
1. Заселяем сначала все самые дешевые отели - сортируем таблицу с комнатами hotels_new по цене
2. Для каждого отеля по порядку: получить список гостей исходя из порядка и приоритета

создать датасет для каждого отеля - временный 
создать совокупный датасет

для каждого отеля отобрать только тех гостей, у которых он встречается в приоритетах
hotel - guest - priority - *rn

отсортировать по приоритету, затем по гостю 
пронумеровать и оставить только тех, кто меньше количества комнат 

также сделать датасеты с заселенными и незаселенными гостями, чтобы в новой итерации обращаться к измененному списку гостей 
то есть датасет с гостями будет меняться

In [1461]:
priorities = []
for i in range(1, max(preferences['priority']) + 1):
    priorities.append(i)

In [1207]:
hotels_new = hotels_new.sort_values(by = ['price', 'hotel_number', 'room_number'])

In [1487]:
hotels = hotels.sort_values('price')

In [ ]:
hotels_draft = hotels.copy()
df_draft = pd.DataFrame()
dict_hotels = hotels_draft.groupby('price')['hotel'].apply(list).to_dict()
checked_in_full = pd.DataFrame(columns = ['guest'])
for key in dict_hotels:
    for hotel in dict_hotels[key]:
        df_draft_0 = preferences[(preferences['hotel'] == hotel) & ~preferences['guest'].isin(checked_in_full['guest'])] #исключаем тех, кого заселили
        df_draft = pd.concat([df_draft, df_draft_0], ignore_index=True)
    for priority in priorities:
        df_priority = df_draft[(df_draft['priority'] == priority) & ~df_draft['guest'].isin(checked_in_full['guest'])] #исключить тех, кого заселили
        for hotel in dict_hotels[key]:
            checked_in_guests = df_priority[df_priority['hotel'] == hotel]
            free_rooms = hotels_draft[hotels_draft['hotel'] == hotel]['rooms'].values[0]
            checked_in_guests = checked_in_guests[:free_rooms]
            checked_in_full = pd.concat([checked_in_full, checked_in_guests], ignore_index=True)
            hotels_draft.loc[hotels_draft['hotel'] == hotel, 'rooms'] -= len(checked_in_guests)
    print(key, dict_hotels[key], len(checked_in_full), 'done')       
    df_draft = pd.DataFrame()

In [1521]:
#check
check = checked_in_full.pivot_table(values= ['guest'], index='hotel', aggfunc={'guest':'count'})
check = check.reset_index()
check = pd.merge(check, hotels, on = 'hotel', how = 'left')
len(check[check['guest'] != check['rooms']]) #53 hotels are not full - too expensive

53

In [1527]:
checked_in_full[checked_in_full['hotel'] == 'hotel_15']

,guest,hotel,priority
153,guest_66,hotel_15,1.0
154,guest_384,hotel_15,1.0
155,guest_471,hotel_15,1.0
156,guest_801,hotel_15,1.0
157,guest_1187,hotel_15,1.0


In [1567]:
leftovers_expensive = preferences[~preferences['guest'].isin(checked_in_full['guest'])]['guest'].unique() #3990 - 10 guests without hotel - why?

In [1573]:
len(leftovers_expensive)

10

In [ ]:
#Check 10 guests and check in them in expensive hotels

In [1591]:
df_for_leftovers_expensive = pd.merge(hotels_new, check[check['guest'] != check['rooms']], left_on = 'hotel_number', right_on = 'hotel', how = 'inner')

In [1593]:
df_for_leftovers_expensive = df_for_leftovers_expensive[df_for_leftovers_expensive['room_number'] <= df_for_leftovers_expensive['guest']]

In [1595]:
df_for_leftovers_expensive = df_for_leftovers_expensive.sort_values('price_y')

In [1597]:
df_for_leftovers_expensive = df_for_leftovers_expensive[:len(leftovers_expensive)]

In [1599]:
df_for_leftovers_expensive['leftovers'] = leftovers_expensive

In [1605]:
df_for_leftovers_expensive = df_for_leftovers_expensive[['leftovers', 'hotel']]

In [1609]:
df_for_leftovers_expensive['priority'] = 0

C:\Users\Mi\AppData\Local\Temp\ipykernel_18604\968085590.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_for_leftovers_expensive['priority'] = 0


In [1623]:
df_for_leftovers_expensive = df_for_leftovers_expensive.rename(columns={'leftovers': 'guest'})

In [1625]:
checked_in_full_final = pd.concat([checked_in_full, df_for_leftovers_expensive])

In [1630]:
checked_in_full_final = pd.merge(checked_in_full_final, guests, on = 'guest', how = 'left')

In [1632]:
checked_in_full_final = pd.merge(checked_in_full_final, hotels, on = 'hotel', how = 'left')

In [1636]:
checked_in_full_final['price_final'] = checked_in_full_final['price'] * (1 - checked_in_full_final['discount'])

In [1638]:
checked_in_group_part3 = checked_in_full_final.pivot_table(values='price_final', index='hotel', aggfunc='sum')

In [1640]:
checked_in_group_part3

,price_final
hotel,
hotel_1,1231.23
hotel_10,2026.86
hotel_100,2734.40
hotel_101,2136.16
hotel_102,1015.43
...,...
hotel_95,237.33
hotel_96,2340.00
hotel_97,757.44


In [1642]:
sum(checked_in_group_part3['price_final'])

581017.41

Part 4

In [ ]:
#sorted by availability starting with the most roomy hotel 

In [1461]:
priorities = []
for i in range(1, max(preferences['priority']) + 1):
    priorities.append(i)

In [1658]:
hotels = hotels.sort_values('rooms', ascending=False)

In [1666]:
hotels_draft = hotels.copy()
df_draft = pd.DataFrame()
dict_hotels = hotels_draft.groupby('rooms')['hotel'].apply(list).to_dict()
checked_in_full = pd.DataFrame(columns = ['guest'])
for key in dict_hotels:
    for hotel in dict_hotels[key]:
        df_draft_0 = preferences[(preferences['hotel'] == hotel) & ~preferences['guest'].isin(checked_in_full['guest'])] #исключаем тех, кого заселили
        df_draft = pd.concat([df_draft, df_draft_0], ignore_index=True)
    for priority in priorities:
        df_priority = df_draft[(df_draft['priority'] == priority) & ~df_draft['guest'].isin(checked_in_full['guest'])] #исключить тех, кого заселили
        for hotel in dict_hotels[key]:
            checked_in_guests = df_priority[df_priority['hotel'] == hotel]
            free_rooms = hotels_draft[hotels_draft['hotel'] == hotel]['rooms'].values[0]
            checked_in_guests = checked_in_guests[:free_rooms]
            checked_in_full = pd.concat([checked_in_full, checked_in_guests], ignore_index=True)
            hotels_draft.loc[hotels_draft['hotel'] == hotel, 'rooms'] -= len(checked_in_guests)
    print(key, dict_hotels[key], len(checked_in_full), 'done')       
    df_draft = pd.DataFrame()

5 ['hotel_87', 'hotel_37', 'hotel_95', 'hotel_15', 'hotel_302', 'hotel_324', 'hotel_85', 'hotel_289', 'hotel_164', 'hotel_244', 'hotel_136', 'hotel_267', 'hotel_307', 'hotel_171', 'hotel_179', 'hotel_357', 'hotel_79', 'hotel_386', 'hotel_235', 'hotel_108', 'hotel_329', 'hotel_64', 'hotel_158', 'hotel_212', 'hotel_119', 'hotel_269', 'hotel_351', 'hotel_258', 'hotel_163', 'hotel_396', 'hotel_291', 'hotel_30', 'hotel_330', 'hotel_358', 'hotel_65', 'hotel_9', 'hotel_188', 'hotel_198', 'hotel_219', 'hotel_131', 'hotel_40'] 205 done
6 ['hotel_62', 'hotel_123', 'hotel_141', 'hotel_292', 'hotel_369', 'hotel_225', 'hotel_391', 'hotel_348', 'hotel_14', 'hotel_382', 'hotel_335', 'hotel_377', 'hotel_216', 'hotel_36', 'hotel_72', 'hotel_157', 'hotel_349', 'hotel_301', 'hotel_55', 'hotel_113', 'hotel_66', 'hotel_51', 'hotel_33', 'hotel_99', 'hotel_107', 'hotel_221', 'hotel_340', 'hotel_210', 'hotel_204', 'hotel_271', 'hotel_203'] 391 done


KeyboardInterrupt: 

In [1521]:
#check
check = checked_in_full.pivot_table(values= ['guest'], index='hotel', aggfunc={'guest':'count'})
check = check.reset_index()
check = pd.merge(check, hotels, on = 'hotel', how = 'left')
len(check[check['guest'] != check['rooms']]) #53 hotels are not full - too expensive

53

In [1537]:
hotels_new[hotels_new['hotel_number'] == 'hotel_1']

,room_number,hotel_number,price,random_number
0,1,hotel_1,273,679
1,2,hotel_1,273,4569
2,3,hotel_1,273,1514
3,4,hotel_1,273,1057
4,5,hotel_1,273,2541
5,6,hotel_1,273,4318
6,7,hotel_1,273,161
7,8,hotel_1,273,2707
8,9,hotel_1,273,3039
9,10,hotel_1,273,4350


In [1519]:
check[check['guest'] != check['rooms']]

,hotel,guest,rooms,price
0,hotel_1,5,13,273
8,hotel_106,1,11,297
19,hotel_117,2,15,282
27,hotel_124,7,10,255
42,hotel_139,2,8,289
51,hotel_147,13,16,251
80,hotel_178,5,8,258
82,hotel_18,1,8,287
87,hotel_185,5,9,273
94,hotel_191,16,17,250


In [1511]:
check2 = checked_in_full.pivot_table( index='guest',  aggfunc={'guest':'count'})

In [1513]:
check2[check2['guest'] == 1] #3990 so cool!

,guest
guest,
guest_1,1
guest_10,1
guest_100,1
guest_1000,1
guest_1001,1
...,...
guest_995,1
guest_996,1
guest_997,1


In [1449]:
len(checked_in_full[checked_in_full['priority'] == 1]) #18

4599

In [1317]:
think_tank = ['hotel_190', 'hotel_49']
df_for_hotel = pd.DataFrame()
df_merged = pd.DataFrame()
not_checked_in_guests = preferences
for hotel in think_tank:
    df_for_hotel = not_checked_in_guests[not_checked_in_guests['hotel'] == hotel]
    df_for_hotel = df_for_hotel.sort_values(by = ['priority']).sort_index(ascending=True)
    df_for_hotel.to_excel(f'output_{hotel}.xlsx', index=False)

In [1345]:
pivot_hotels = hotels.pivot_table(values='hotel', index='price', aggfunc='count')

Все, что дальше - не нужно